# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import re
from sqlalchemy import create_engine
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM Disaster_message", engine)
X = df["message"]
Y = df.iloc[: ,4:]

### 2. Write a tokenization function to process your text data

In [3]:
nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
def tokenize(text):
    """
    a function that takes text and return a clean token
    input : text
    output : a clean token
    """
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

In [8]:

category_names = list(df.columns[4:])

for i in range(len(category_names)):
    print("Category:", category_names[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))

Category: related 
              precision    recall  f1-score   support

          0       0.65      0.35      0.46      1516
          1       0.82      0.94      0.88      4989
          2       0.50      0.18      0.27        49

avg / total       0.78      0.80      0.78      6554

Category: request 
              precision    recall  f1-score   support

          0       0.88      0.99      0.93      5461
          1       0.83      0.35      0.49      1093

avg / total       0.88      0.88      0.86      6554

Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

Category: aid_related 
              precision    recall  f1-score   support

          0       0.73      0.88      0.80      3843
          1       0.76      0.53      0.62      2711

avg / total       0.74      0.74      0.73      6554

Categ

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
pipeline_adaboost.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f54b47d09d8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=0.5, n_estimators=70, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [10]:
parameters =  {
    'vect__max_df': (0.5, 0.75, 1.0),
    'clf__estimator__warm_start': [False,True],
    'clf__estimator__min_samples_split': [2,3],
    'clf__estimator__criterion': ['gini', 'entropy'],
    }


cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
cv.fit(X_train,y_train)
y_pred = cv.predict(X_test)

In [12]:
print(cv.best_params_)

{'clf__estimator__criterion': 'gini', 'clf__estimator__min_samples_split': 2, 'clf__estimator__warm_start': True, 'vect__max_df': 0.5}


In [13]:
accuracy = (y_pred == y_test).sum()/y_test.shape[0]
print('accuracy :\n', accuracy)
accuracy.sum()

accuracy :
 related                   0.792951
request                   0.886634
offer                     0.996033
aid_related               0.732072
medical_help              0.923711
medical_products          0.954074
search_and_rescue         0.971926
security                  0.982911
military                  0.967958
child_alone               1.000000
water                     0.947818
food                      0.914098
shelter                   0.918676
clothing                  0.987641
money                     0.978334
missing_people            0.990388
refugees                  0.965822
death                     0.957736
other_aid                 0.872902
infrastructure_related    0.934086
transport                 0.953921
buildings                 0.949344
electricity               0.978639
tools                     0.992829
hospitals                 0.986573
shops                     0.996338
aid_centers               0.988099
other_infrastructure      0.956057
weather_

33.922032346658533

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [5]:
pipeline_adaboost = Pipeline([
        ('vect',CountVectorizer(tokenizer = tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators=70, learning_rate = 0.5)))
])

In [ ]:
pipeline_adaboost.fit(X_train,y_train)

In [15]:
y_pred = pipeline_adaboost.predict(X_test)

In [16]:
category_names = list(df.columns[4:])

for i in range(len(category_names)):
    print("Category:", category_names[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))

Category: related 
              precision    recall  f1-score   support

          0       0.64      0.13      0.22      1516
          1       0.78      0.98      0.87      4989
          2       0.20      0.02      0.04        49

avg / total       0.75      0.77      0.71      6554

Category: request 
              precision    recall  f1-score   support

          0       0.91      0.98      0.94      5461
          1       0.82      0.52      0.63      1093

avg / total       0.89      0.90      0.89      6554

Category: offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

Category: aid_related 
              precision    recall  f1-score   support

          0       0.75      0.88      0.81      3843
          1       0.78      0.58      0.67      2711

avg / total       0.76      0.76      0.75      6554

Categ

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
accuracy = (y_pred == y_test).sum()/y_test.shape[0]
print('accuracy :\n', accuracy)
accuracy.sum()

accuracy :
 related                   0.774947
request                   0.899756
offer                     0.995880
aid_related               0.758316
medical_help              0.927220
medical_products          0.959414
search_and_rescue         0.973604
security                  0.981843
military                  0.972231
child_alone               1.000000
water                     0.961398
food                      0.942936
shelter                   0.943241
clothing                  0.989625
money                     0.980928
missing_people            0.991303
refugees                  0.967958
death                     0.968569
other_aid                 0.872749
infrastructure_related    0.935154
transport                 0.958804
buildings                 0.956668
electricity               0.981080
tools                     0.992829
hospitals                 0.986421
shops                     0.996338
aid_centers               0.987794
other_infrastructure      0.956668
weather_

34.121605126640219

### 9. Export your model as a pickle file

In [22]:
import pickle
pickle.dump(pipeline_adaboost.best_estimator_, open(model_filepath, 'wb'))

AttributeError: 'Pipeline' object has no attribute 'best_estimator_'

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.